In [1]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "8g")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/10/01 20:43:48 WARN Utils: Your hostname, AshwinHPEnvy resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/01 20:43:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/01 20:43:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
revenue_sdf = (spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("delimiter", ",")   
    .csv("../data/curated/revenue.csv"))

customer_sdf = spark.read.parquet("../data/top_200_customer/*")

trans_final_sdf = spark.read.parquet('../data/merged/merged_transactions_with_tags.parquet')


In [3]:
revenue_sdf.show(5, truncate=False)

customer_sdf.show(5, truncate=False)

trans_final_sdf.show(5, truncate=False)

+------------+----+----+-------------+---------------------------------------------------------------------------------+------------------+----------+------------+-----------------+---------+-------------+--------------------+----------------------------------+----------------------------------+-----------+------------------+
|merchant_abn|week|year|merchant_name|description                                                                      |revenue           |male_count|female_count|undisclosed_count|num_trans|num_postcodes|num_unique_customers|average_consumer_fraud_probability|average_merchant_fraud_probability|fraud_count|mean_trans_val    |
+------------+----+----+-------------+---------------------------------------------------------------------------------+------------------+----------+------------+-----------------+---------+-------------+--------------------+----------------------------------+----------------------------------+-----------+------------------+
|10023283211 |8 

24/10/01 20:44:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------------------------------------------------+-------+------------+------------------+------------------------------------+--------------+-------------------------+-----------+--------------+---------------------------+-----+--------+------+--------------------------+--------------------------+--------------------+------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------+----------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------+------------+--------------------+--------------------------+--------------------+--------+------------+---------+----------------+
|description           

In [4]:
revenue_sdf.printSchema()
customer_sdf.printSchema()

root
 |-- merchant_abn: long (nullable = true)
 |-- week: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- revenue: double (nullable = true)
 |-- male_count: integer (nullable = true)
 |-- female_count: integer (nullable = true)
 |-- undisclosed_count: integer (nullable = true)
 |-- num_trans: integer (nullable = true)
 |-- num_postcodes: integer (nullable = true)
 |-- num_unique_customers: integer (nullable = true)
 |-- average_consumer_fraud_probability: double (nullable = true)
 |-- average_merchant_fraud_probability: double (nullable = true)
 |-- fraud_count: integer (nullable = true)
 |-- mean_trans_val: double (nullable = true)

root
 |-- merchant_abn: long (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- total_cumulative_customers: integer (nullable = true)
 |-- total_new_unique_customers: long (nullable = true)



In [5]:
trans_final_sdf.printSchema()

root
 |-- description: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- merchant_abn: long (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: date (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- consumer_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- consumer_fraud_probability: double (nullable = true)
 |-- merchant_fraud_probability: double (nullable = true)
 |-- LOCALITY_NAME: string (nullable = true)
 |-- list_SA2_CODE: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- list_ERP: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- list_num_earners: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- list_total_income_($M): a

In [6]:
# Find the latest order_datetime
latest_date = customer_sdf.select(F.max("order_datetime")).collect()[0][0]

# Filter the dataframe to get entries with the latest order_datetime
latest_customers_df = customer_sdf.filter(customer_sdf["order_datetime"] == latest_date)

# Select the required columns: merchant_abn, total_cumulative_customers
latest_customers_df.select("merchant_abn", "total_cumulative_customers").show()

+------------+--------------------------+
|merchant_abn|total_cumulative_customers|
+------------+--------------------------+
| 63290521567|                     24068|
| 80324045558|                     24076|
| 68216911708|                     24065|
| 21439773999|                     23914|
| 89726005175|                     24079|
| 64403598239|                     23875|
| 43186523025|                     24076|
| 72472909171|                     23747|
| 45629217853|                     24081|
| 79417999332|                     23731|
| 94493496784|                     23660|
| 60956456424|                     23499|
| 32361057556|                     23396|
| 46804135891|                     24079|
| 91923722701|                     23193|
| 96152467973|                     22661|
| 57900494384|                     22627|
| 82065156333|                     22336|
| 49505931725|                     22686|
| 19933438190|                     22671|
+------------+--------------------

In [7]:
latest_date_cust = customer_sdf.select(F.max("order_datetime")).collect()[0][0]
latest_date_trans = trans_final_sdf.select(F.max("order_datetime")).collect()[0][0]

print([latest_date_cust, latest_date_trans])

[datetime.date(2022, 10, 26), datetime.date(2022, 10, 26)]


In [20]:
from pyspark.sql.window import Window

# Filter merchants from customer_sdf
merchant_abn_list = customer_sdf.select("merchant_abn").distinct()

# 1) Get Take Rate and Industry Segment from trans_final_sdf
trans_data = trans_final_sdf \
    .select("merchant_abn", "take_rate", "industry_segment") \
    .join(merchant_abn_list, "merchant_abn", "inner") \
    .dropDuplicates(["merchant_abn"])

# 2) Calculate Fraud Rate from revenue_sdf
revenue_data = revenue_sdf \
    .withColumn("fraud_rate", F.col("fraud_count") / F.col("num_trans")) \
    .select("merchant_abn", "fraud_rate") \
    .join(merchant_abn_list, "merchant_abn", "inner") \
    .dropDuplicates(["merchant_abn"])


# 3) Cumulative Revenue and Transactions over the Last 6 Months from revenue_sdf
six_months_ago = F.add_months(F.lit(latest_date), -6)

cumulative_data = trans_final_sdf \
    .filter(F.col("order_datetime") > six_months_ago) \
    .groupBy("merchant_abn") \
    .agg(
        F.sum("dollar_value").alias("cumulative_revenue_6m"),
        F.count("order_id").alias("cumulative_transactions_6m")
    ) \
    .join(merchant_abn_list, "merchant_abn", "inner") \


# 4) Total Customers at Latest Date in customer_sdf
latest_customers = customer_sdf \
    .filter(customer_sdf["order_datetime"] == latest_date) \
    .select("merchant_abn", "total_cumulative_customers")

# 5) Revenue Growth (compare last month vs. 12 months ago)

# trans_final_sdf = trans_final_sdf \
#    .withColumn("year", F.year("order_datetime")) \
#    .withColumn("month", F.month("order_datetime"))

# window_spec = Window.partitionBy("merchant_abn").orderBy("year", "month")

# revenue_growth = trans_final_sdf \
#    .withColumn("last_month", F.sum("dollar_value").over(window_spec)) \
#    .withColumn("prev_year", F.lag("last_month", 12).over(window_spec)) \
#    .withColumn("revenue_growth", (F.col("last_month") - F.col("prev_year")) / F.col("prev_year")) \
#    .select("merchant_abn", "revenue_growth") /*

# 6) Calculate Industry Segment Popularity from trans_final_sdf

total_merchants = trans_final_sdf.select("merchant_abn").distinct().count()

industry_segment_popularity = trans_final_sdf \
    .groupBy("industry_segment") \
    .agg(F.countDistinct("merchant_abn").alias("num_merchants")) \
    .withColumn("segment_popularity", F.col("num_merchants") / total_merchants) \
    .select("industry_segment", "segment_popularity")

print(merchant_abn_list.select("merchant_abn").count())

print(trans_data.select("merchant_abn").count())

print("Revenue Data:")
revenue_data.show(5, truncate=False)
print(revenue_data.select("merchant_abn").count())

print("Cumulative Data:")
cumulative_data.show(5, truncate=False)
print(cumulative_data.select("merchant_abn").count())

print("Latest Customers:")
latest_customers.show(5, truncate=False)
print(latest_customers.select("merchant_abn").count())


print("Industry Segment Popularity:")
industry_segment_popularity.show(5, truncate=False)

200


200
Revenue Data:


+------------+----------+
|merchant_abn|fraud_rate|
+------------+----------+
|10648956813 |0.0       |
|15560455575 |0.0       |
|16644129035 |0.0       |
|14492521225 |0.0       |
|11439466003 |0.0       |
+------------+----------+
only showing top 5 rows



200
Cumulative Data:


+------------+---------------------+--------------------------+
|merchant_abn|cumulative_revenue_6m|cumulative_transactions_6m|
+------------+---------------------+--------------------------+
|83412691377 |167701.22576018586   |4853                      |
|10648956813 |472832.5424789102    |7329                      |
|86662713230 |375492.4969336751    |7141                      |
|62191208634 |477967.0756249253    |5489                      |
|27093785141 |3230303.510783694    |8677                      |
+------------+---------------------+--------------------------+
only showing top 5 rows



200
Latest Customers:
+------------+--------------------------+
|merchant_abn|total_cumulative_customers|
+------------+--------------------------+
|63290521567 |24068                     |
|80324045558 |24076                     |
|68216911708 |24065                     |
|21439773999 |23914                     |
|89726005175 |24079                     |
+------------+--------------------------+
only showing top 5 rows



200
Industry Segment Popularity:


+---------------------------+--------------------+
|industry_segment           |segment_popularity  |
+---------------------------+--------------------+
|rental services            |0.033283656234475906|
|retail                     |0.6418281172379533  |
|luxury products + services |0.12468951813214109 |
|health + beauty            |0.07824143070044709 |
|technology + media services|0.12195727769498262 |
+---------------------------+--------------------+



In [21]:
# 7) Combine all into final DataFrame
final_df = trans_data \
    .join(revenue_data, "merchant_abn", "inner") \
    .join(cumulative_data, "merchant_abn", "inner") \
    .join(latest_customers, "merchant_abn", "inner") \
    .join(industry_segment_popularity, "industry_segment", "inner") \
#    .join(revenue_growth, "merchant_abn", "inner") \

print(final_df.select("merchant_abn").count())

200


In [29]:
# Define a list of columns to normalize (excluding 'merchant_abn')
columns_to_normalize = final_df.columns
columns_string = ["merchant_abn", "industry_segment"]
columns_to_normalize = list(set(columns_to_normalize) - set(columns_string))

# Calculate min and max for each column
min_max_values = {col: final_df.agg(F.min(col).alias("min"), F.max(col).alias("max")).first() for col in columns_to_normalize}

# Create a new DataFrame with normalized values
df_normalized = final_df

for col in columns_to_normalize:
    min_val = min_max_values[col]["min"]
    max_val = min_max_values[col]["max"]
    
    # Normalize the column
    df_normalized = df_normalized.withColumn(
        f"{col}_normalized",
        (F.col(col) - min_val) / (max_val - min_val)
    )

In [34]:
df_normalized \
    .select("take_rate", "take_rate_normalized") \
    .orderBy("take_rate") \
    .show(20, truncate=False)


+---------+--------------------+
|take_rate|take_rate_normalized|
+---------+--------------------+
|0.16     |0.0                 |
|0.22     |0.00879765395894428 |
|0.38     |0.03225806451612903 |
|1.44     |0.187683284457478   |
|1.47     |0.19208211143695014 |
|1.47     |0.19208211143695014 |
|1.47     |0.19208211143695014 |
|1.52     |0.1994134897360704  |
|1.55     |0.20381231671554254 |
|1.57     |0.20674486803519063 |
|1.57     |0.20674486803519063 |
|1.61     |0.2126099706744868  |
|1.62     |0.21407624633431088 |
|1.67     |0.22140762463343108 |
|1.67     |0.22140762463343108 |
|1.76     |0.23460410557184752 |
|1.83     |0.24486803519061584 |
|1.85     |0.24780058651026393 |
|1.86     |0.249266862170088   |
|1.87     |0.250733137829912   |
+---------+--------------------+
only showing top 20 rows



In [32]:
import numpy as np

# Apply weights
weights = {
    'take_rate': 0.31,
    'fraud_rate': 0.25,
    "cumulative_revenue_6m": 0.21,
    "cumulative_transactions_6m": 0.08,
    "total_cumulative_customers": 0.08,
    "segment_popularity": 0.07,
    # Add other criteria and weights
}

df_weighted = df_normalized.withColumn(
    'weighted_fraud_rate', F.col('take_rate_normalized') * weights['fraud_rate']
).withColumn(
    'weighted_take_rate', F.col('fraud_rate_normalized') * weights['take_rate']
).withColumn(
    'weighted_revenue', F.col('cumulative_revenue_6m_normalized') * weights['cumulative_revenue_6m']
).withColumn(
    'weighted_transactions', F.col('cumulative_transactions_6m_normalized') * weights['cumulative_transactions_6m']
).withColumn(
    'weighted_customers', F.col('total_cumulative_customers_normalized') * weights['total_cumulative_customers']
).withColumn(
    'weighted_segment', F.col('segment_popularity_normalized') * weights['segment_popularity']
)

# Compute Ideal and Negative Ideal Solutions
ideal_solution = {
    'fraud_rate': df_weighted.agg(F.min('weighted_fraud_rate')).collect()[0][0],
    'take_rate': df_weighted.agg(F.max('weighted_take_rate')).collect()[0][0],
    'revenue': df_weighted.agg(F.max('weighted_revenue')).collect()[0][0],
    'transactions': df_weighted.agg(F.max('weighted_transactions')).collect()[0][0],
    'customers': df_weighted.agg(F.max('weighted_customers')).collect()[0][0],
    'segment': df_weighted.agg(F.min('weighted_segment')).collect()[0][0]
}

negative_ideal_solution = {
    'fraud_rate': df_weighted.agg(F.max('weighted_fraud_rate')).collect()[0][0],
    'take_rate': df_weighted.agg(F.min('weighted_take_rate')).collect()[0][0],
    'revenue': df_weighted.agg(F.min('weighted_revenue')).collect()[0][0],
    'transactions': df_weighted.agg(F.min('weighted_transactions')).collect()[0][0],
    'customers': df_weighted.agg(F.min('weighted_customers')).collect()[0][0],
    'segment': df_weighted.agg(F.max('weighted_segment')).collect()[0][0]
}

# Compute distances from Ideal and Negative Ideal Solutions
df_topsis = df_weighted.withColumn(
    'D_positive', F.sqrt(
        (F.col('weighted_fraud_rate') - ideal_solution['fraud_rate']) ** 2 +
        (F.col('weighted_take_rate') - ideal_solution['take_rate']) ** 2 +
        (F.col('weighted_revenue') - ideal_solution['revenue']) ** 2 +
        (F.col('weighted_transactions') - ideal_solution['transactions']) ** 2 +
        (F.col('weighted_customers') - ideal_solution['customers']) ** 2 +
        (F.col('weighted_segment') - ideal_solution['segment']) ** 2
    )
).withColumn(
    'D_negative', F.sqrt(
        (F.col('weighted_fraud_rate') - negative_ideal_solution['fraud_rate']) ** 2 +
        (F.col('weighted_take_rate') - negative_ideal_solution['take_rate']) ** 2 +
        (F.col('weighted_revenue') - negative_ideal_solution['revenue']) ** 2 + 
        (F.col('weighted_transactions') - negative_ideal_solution['transactions']) ** 2 +
        (F.col('weighted_customers') - negative_ideal_solution['customers']) ** 2 +
        (F.col('weighted_segment') - negative_ideal_solution['segment']) ** 2

    )
)

# Compute closeness and rank merchants
df_topsis = df_topsis.withColumn(
    'closeness', F.col('D_negative') / (F.col('D_positive') + F.col('D_negative'))
).orderBy(F.desc('closeness'))

df_topsis.show(5, truncate = False)


+----------------+------------+---------+--------------------+---------------------+--------------------------+--------------------------+-------------------+-----------------------------+--------------------------------+---------------------+-------------------------------------+-------------------------------------+--------------------+--------------------+------------------+-------------------+---------------------+-------------------+----------------+-------------------+-------------------+-------------------+
|industry_segment|merchant_abn|take_rate|fraud_rate          |cumulative_revenue_6m|cumulative_transactions_6m|total_cumulative_customers|segment_popularity |segment_popularity_normalized|cumulative_revenue_6m_normalized|fraud_rate_normalized|cumulative_transactions_6m_normalized|total_cumulative_customers_normalized|take_rate_normalized|weighted_fraud_rate |weighted_take_rate|weighted_revenue   |weighted_transactions|weighted_customers |weighted_segment|D_positive         |D

In [35]:
df_weighted \
    .select("take_rate", "take_rate_normalized", "weighted_take_rate") \
    .orderBy("take_rate") \
    .show(20, truncate=False)


+---------+--------------------+------------------+
|take_rate|take_rate_normalized|weighted_take_rate|
+---------+--------------------+------------------+
|0.16     |0.0                 |0.0               |
|0.22     |0.00879765395894428 |0.0               |
|0.38     |0.03225806451612903 |0.0               |
|1.44     |0.187683284457478   |0.0               |
|1.47     |0.19208211143695014 |0.0               |
|1.47     |0.19208211143695014 |0.0               |
|1.47     |0.19208211143695014 |0.0               |
|1.52     |0.1994134897360704  |0.0               |
|1.55     |0.20381231671554254 |0.0               |
|1.57     |0.20674486803519063 |0.0               |
|1.57     |0.20674486803519063 |0.0               |
|1.61     |0.2126099706744868  |0.0               |
|1.62     |0.21407624633431088 |0.0               |
|1.67     |0.22140762463343108 |0.0               |
|1.67     |0.22140762463343108 |0.0               |
|1.76     |0.23460410557184752 |0.0               |
|1.83     |0

In [37]:
window_spec = Window.partitionBy("industry_segment").orderBy(F.desc("closeness"))  # Replace 'closeness' with your ranking metric

ranked_df = df_topsis \
    .withColumn("rank", F.row_number().over(window_spec))

top_merchants = ranked_df.filter(F.col("rank") <= 5)

top_merchants.select("merchant_abn", "industry_segment", "closeness", "rank").show(truncate=False)


+------------+---------------------------+-------------------+----+
|merchant_abn|industry_segment           |closeness          |rank|
+------------+---------------------------+-------------------+----+
|46804135891 |health + beauty            |0.7671359597560261 |1   |
|18158387243 |health + beauty            |0.4516591370478957 |2   |
|11237511112 |health + beauty            |0.37710088475872605|3   |
|48534649627 |health + beauty            |0.3625273853009146 |4   |
|11566786699 |health + beauty            |0.3284626376612607 |5   |
|34920839316 |luxury products + services |0.404465392767599  |1   |
|86578477987 |luxury products + services |0.3842066209081606 |2   |
|76626119831 |luxury products + services |0.37847280657495075|3   |
|68559320474 |luxury products + services |0.37061028095284076|4   |
|23338656015 |luxury products + services |0.36336620468811903|5   |
|41944909975 |retail                     |0.48684134201102414|1   |
|39649557865 |retail                     |0.4655

In [43]:
top_merchants_df = df_topsis \
    .orderBy(F.desc("closeness")) \
    .limit(25)  


industry_count = top_merchants_df \
    .groupBy("industry_segment") \
    .agg(F.count("merchant_abn").alias("merchant_count"))

total_top_merchants = 25 
industry_proportion = industry_count \
    .withColumn("proportion", F.col("merchant_count") / total_top_merchants)

industry_proportion.select("industry_segment", "merchant_count", "proportion").show(truncate=False)

+---------------------------+--------------+----------+
|industry_segment           |merchant_count|proportion|
+---------------------------+--------------+----------+
|health + beauty            |2             |0.08      |
|retail                     |19            |0.76      |
|luxury products + services |1             |0.04      |
|technology + media services|3             |0.12      |
+---------------------------+--------------+----------+



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `merchant_count` cannot be resolved. Did you mean one of the following? [`overall_merchant_count`, `industry_segment`, `overall_proportion`].;
'Project [industry_segment#85, 'merchant_count, 'proportion]
+- Project [industry_segment#85, overall_merchant_count#7721L, (cast(overall_merchant_count#7721L as double) / cast(4026 as double)) AS overall_proportion#7731]
   +- Aggregate [industry_segment#85], [industry_segment#85, count(merchant_abn#59L) AS overall_merchant_count#7721L]
      +- Project [description#57, user_id#58L, merchant_abn#59L, dollar_value#60, order_id#61, order_datetime#62, merchant_name#63, consumer_id#64L, name#65, address#66, state#67, postcode#68, gender#69, consumer_fraud_probability#70, merchant_fraud_probability#71, LOCALITY_NAME#72, list_SA2_CODE#73, list_ERP#74, list_num_earners#75, list_total_income_($M)#76, list_mean_income#77, postcode_ERP#78, postcode_num_earners